In [ ]:
# coding: utf-8

# In[1]:


import json
import csv
import itertools
import pandas as pd
import numpy as np


# In[2]:


# import ijson

# events_data = []
# filename = "data.json"
# # with open(filename, 'r') as f:
# #     objects = ijson.items(f, '')
#
# #     for row in objects:
# #         events_data.append(row)
#
#
# # In[3]:
#
#
# user_events = pd.read_json(filename)
#
#
# # In[4]:
#
#
# user_events.head()
#
#
# # In[5]:
#
#
# user_dict = {}
# for index, each in user_events.iterrows():
#     if each["distinct_id"] not in user_dict:
#         user_dict[each["distinct_id"]] = [{each["event"]: [each["time"], each["parent_id"]]}]
#     else:
#         user_dict[each["distinct_id"]].append({each["event"]: [each["time"], each["parent_id"]]})
#
#
# # In[6]:
#
#
# # with open('user_event_data_21-02-2018.json', 'wt') as jsonfile:
# #     json.dump(user_dict, jsonfile)
#
#
# # In[7]:
#
#
# user_dict


In [ ]:
json_data = open("user_event_data_21-02-2018.json")
user_dict = json.load(json_data)

# In[16]:


female_lookup = open("parent_item_data_female_26-02-2018.json")

look_up_f = json.load(female_lookup)

male_lookup = open("parent_item_data_male_26-02-2018.json")

look_up_m = json.load(male_lookup)


look_up = look_up_f.copy()   # start with x's keys and values
look_up.update(look_up_m)
#look_up = look_up_f + look_up_m


##for each in look_up:
##    print(each,look_up[each])


# In[ ]:


In [ ]:
## Lookup list for events
unique_dict = {}
user_event_dict_with_params = {}
#user_dict_dummy = {'1573d7c483eb3-06235568306ea58-4c594331-100200-1573d7c484011c': [{'Add to Cart Web': [1503533058,'FRES05834']}]}
for user, evnt_obj in user_dict.items():
    user_event_dict_with_params[user] = {}
    unique_dict = {"Add to Cart Web": [], "Details Web": []}

    for event in evnt_obj:
        key, value = list(event.keys())[0], list(event.values())[0]
        #print(key, value)
        params = []
        try:
            if look_up[value[1]]:
                if key == "Add to Cart Web" and value[1] not in unique_dict[key]:
                    params = [value[1], look_up[value[1]]["inventory_category"], look_up[value[1]]["brand_id"], look_up[value[1]]["color"], look_up[value[1]]["rental"]]
                    unique_dict[key].append(value[1])
                elif key == "Details Web" and value[1] not in unique_dict[key]:
                    params = [value[1], look_up[value[1]]["inventory_category"], look_up[value[1]]["brand_id"], look_up[value[1]]["color"], look_up[value[1]]["rental"]]
                    unique_dict[key].append(value[1])


                if params != []:
                    if key in user_event_dict_with_params[user].keys():
                        user_event_dict_with_params[user][key].append(params)
                    else:
                        user_event_dict_with_params[user][key] = [params]

        except KeyError:
            pass

In [ ]:
# In[ ]:


## User Affinities Dictionary
userwise_total = {}
for user in user_event_dict_with_params:
    atc_count = 0
    detail_count = 0
    for event, val in user_event_dict_with_params[user].items():
        #for key, value in val.items():
        if event == "Add to Cart Web":
            atc_count = val.__len__()
        elif event == "Details Web":
            detail_count = val.__len__()
    if user in userwise_total.keys():
        userwise_total[user]["Add to Cart Web"] += atc_count
        userwise_total[user]["Details Web"] += detail_count
    else:
        userwise_total[user] = {"Add to Cart Web": atc_count ,
                                "Details Web": detail_count}
    # if "Add to Cart Web" in userwise_total[user].keys():
    #     userwise_total[user]["Add to Cart Web"] = atc_count
    # else:
    #     userwise_total[user] = {"Add to Cart Web":  atc_count}
    #
    # if "Details Web" in userwise_total[user].keys():
    #     userwise_total[user]["Details Web"] = detail_count
    # else:
    #     userwise_total[user] = {"Details Web":  detail_count}


In [ ]:
user_aff = {}
par_inv = {}
for user in user_event_dict_with_params:
    if user not in user_aff.keys():
        user_aff[user] = {"Add to Cart Web": {"inv_cat": {}, "brand": {}, "color_tone": {}}, "Details Web": {"inv_cat": {}, "brand": {}, "color_tone": {}}}
    for event, values in user_event_dict_with_params[user].items():
        #for key, value in event.items():
        for val in values:
            if len(val) == 5:
                par_inv[val[0]] = val[1]
            try:

                if val[1] not in user_aff[user][event]["inv_cat"]:
                    user_aff[user][event]["inv_cat"][val[1]] = 1
                else:
                    user_aff[user][event]["inv_cat"][val[1]] += 1

                if val[2] not in user_aff[user][event]["brand"]:
                    user_aff[user][event]["brand"][val[2]] = 1
                else:
                    user_aff[user][event]["brand"][val[2]] += 1

                if val[3] not in user_aff[user][event]["color_tone"]:
                    user_aff[user][event]["color_tone"][val[3]] = 1
                else:
                    user_aff[user][event]["color_tone"][val[3]] += 1
            except IndexError:
                pass



In [ ]:
# In[ ]:


## Normalising the user affinities

for user in user_aff:
    for key in user_aff[user]:
        #for key, value in event.items():
        if key == "Add to Cart Web":
            for each in user_aff[user][key]["inv_cat"]:
                user_aff[user][key]["inv_cat"][each] = user_aff[user][key]["inv_cat"][each]/userwise_total[user]["Add to Cart Web"]
            for each in user_aff[user][key]["brand"]:
                user_aff[user][key]["brand"][each] = user_aff[user][key]["brand"][each]/userwise_total[user]["Add to Cart Web"]
            for each in user_aff[user][key]["color_tone"]:
                user_aff[user][key]["color_tone"][each] = user_aff[user][key]["color_tone"][each]/userwise_total[user]["Add to Cart Web"]
        elif key == "Details Web":
            for each in user_aff[user][key]["inv_cat"]:
                user_aff[user][key]["inv_cat"][each] = user_aff[user][key]["inv_cat"][each]/userwise_total[user]["Details Web"]
            for each in user_aff[user][key]["brand"]:
                user_aff[user][key]["brand"][each] = user_aff[user][key]["brand"][each]/userwise_total[user]["Details Web"]
            for each in user_aff[user][key]["color_tone"]:
                user_aff[user][key]["color_tone"][each] = user_aff[user][key]["color_tone"][each]/userwise_total[user]["Details Web"]



In [ ]:

# In[ ]:


## Updating User_aff with High roller affinity

for user in user_event_dict_with_params:
    user_avg_rental_atc = 0
    user_avg_rental_dw = 0
    for event, values in user_event_dict_with_params[user].items():
        for value in values:
            if value.__len__() != 0:
                if event == "Add to Cart Web":
                    user_avg_rental_atc += value[4]/userwise_total[user]["Add to Cart Web"]
                elif event == "Details Web":
                    user_avg_rental_dw += value[4]/userwise_total[user]["Details Web"]
    if user_avg_rental_atc > 2500:
        user_aff[user]["Add to Cart Web"]["high_roller"] = 1
    else :
        user_aff[user]["Add to Cart Web"]["high_roller"] = user_avg_rental_atc/2500
    if user_avg_rental_dw > 2500:
        user_aff[user]["Details Web"]["high_roller"] = 1
    else :
        user_aff[user]["Details Web"]["high_roller"] = user_avg_rental_dw/2500



In [ ]:
with open('user_affinity.json', 'w') as f:
    f.write(json.dumps(user_aff))